In [0]:
from pyspark.sql.functions import col, count
from pyspark.sql.types import IntegerType, DoubleType
import mlflow
import mlflow.spark
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.regression import RandomForestRegressor